# Proof of concept
## Get Groundstate of H2 with AdaptVqe
- taken from: https://qiskit-community.github.io/qiskit-nature/howtos/adapt_vqe.html

In [1]:
# imports
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

from qiskit_algorithms import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

## Define problem with ansatz

In [2]:
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
problem = driver.run()

In [3]:
mapper = JordanWignerMapper()

In [4]:
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [5]:
# define vqe
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

## now setup AdaptVQE

In [6]:
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

## Now get some results

In [7]:
solver = GroundStateEigensolver(mapper, adapt_vqe)

In [8]:
result = solver.solve(problem)

print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Total ground state energy = -1.1373
